In [5]:
# Machine Learning ile Takeoff Weight hesaplanmaktadır. 
# Uçuşun Climb fazındaki featureler kullanılarak model oluşturulmuştur. 

import pandas as pd
from aircraftLib import JetAircraft
from UtilityFunctions import *
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import os
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import style
import time
from sklearn import preprocessing, model_selection, svm, neighbors
from sklearn.linear_model import LinearRegression, Lasso
from scipy import optimize
from tsfresh import extract_features, extract_relevant_features, select_features
from sklearn.neighbors import KNeighborsRegressor
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_relevant_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor,ExtraTreesRegressor
from sklearn.metrics import classification_report
from tsfresh.feature_extraction import feature_calculators, settings
from tsfresh.utilities.distribution import MultiprocessingDistributor
from tsfresh.utilities.dataframe_functions import impute_dataframe_range
from sklearn.metrics import accuracy_score



In [8]:
style.use('ggplot')

basepath = "C:/Users/Recep/Desktop/Boeing Proje/"  ## Configure this line
odp = 'B777_Data'  ## Airport Pairs
datapath = basepath + odp

files = [f for f in listdir(datapath) if isfile(join(datapath, f))]
fig1 = plt.figure(1,figsize=[7, 5])
ac = JetAircraft('B773ERGE115B')

constants = {}
constants["lb_to_kg"] = 0.45359
constants["hr_to_sec"] = 1/3600
constants["m_to_nm"] = 0.0005399568
constants["knot_to_ms"] = 0.51444
constants["ft_to_m"] = 0.3048 

knot_to_ms = constants["knot_to_ms"]
ft_to_m = constants["ft_to_m"]
g0 = 9.8065
g0_feet = 32.1741

<Figure size 504x360 with 0 Axes>

In [68]:
from cmath import nan

counter = 0
y = []
specificEnergyGradient = []
specificEnergy = []
flightPathAngle = []
distance_from_takeoff_list = []
landing_gear = []


df = pd.DataFrame()
for file in files:
    counter +=1
    """ if counter == 4:
        break """
    filepath = os.path.join(odp, file)
    data = pd.read_csv(filepath, low_memory=False, skiprows=1)
    data.rename(columns={'Unnamed: 0':'Time'}, inplace=True)
    data['id'] = np.full((len(data),1), counter-1)
    mass_0 = data.aZFW[0] * constants['lb_to_kg'] + data.aTFQ[0]
    fuel_flow = (data.aFF1+data.aFF2)*constants["lb_to_kg"]*constants["hr_to_sec"]
    

    data.drop(columns=['aGW','aWNDDIR','aWNDSPD','aSPDBRK','TRUEHEADNG','aFF1','aFF2','aTFQ','aZFW','WFCMD'], axis=1,inplace=True)
    
    y.append(mass_0)
    #df = df.append(data, ignore_index=True)

    distance_from_takeoff = 0
    fuel_consumption = 0 
    mass = []
    takeoff_alt_list = []
    takeoff_tas_list = []
    sigma = pressureRatio(data.aSAP)
    theta = temperatureRatio(data.aSAT)
    isaDev = isaDeviation(data.aALTBARO, data.aSAT)
    tas = cas2tas(data.aCAS, data.aALTBARO, isaDev)
    j=0

    for i in range(len(data)-1):
        
        if data.aLDGDSEL[i] == "DOWN":
            landing_gear.append(1)
        else:
            landing_gear.append(0)
        
        horizontalSpeed = haversine((data.LATPOS[i],data.LONPOS[i]),(data.LATPOS[i+1],data.LONPOS[i+1]))
        distance_from_takeoff = distance_from_takeoff + haversine((data.LATPOS[i],data.LONPOS[i]),(data.LATPOS[i+1],data.LONPOS[i+1])) * constants["m_to_nm"]
        flightMode, ROCD = ac.identifyFlightMode(data.aALTBARO[i+1],data.aALTBARO[i])

        fuel_consumption += fuel_flow[i]
        mass.append(mass_0 - fuel_consumption)         

        drag = ac.calculate_drag(mass[i],sigma[i],data.aMACH[i],0)
        j+=1
        #if np.abs(distance_from_takeoff) > 5:
        if data.aALTBARO[i] > 1500:
            j-=1
            
            Vdot = (tas[i+1]-tas[i]) * ft_to_m
            distance_from_takeoff_list.append(distance_from_takeoff)
            flightPathAngle.append(np.arctan2(ROCD,horizontalSpeed))
            specEnergyGrad = tas[i] * ft_to_m * Vdot + g0 * ROCD * ft_to_m
            specificEnergyGradient.append(specEnergyGrad)
            specificEnergy.append(0.5*((tas[i]*ft_to_m)**2) + g0 * data.aALTBARO[i] * ft_to_m)    
            #if np.abs(distance_from_takeoff) > 50:    # Distance to Takeoff
            #df = df.append(data.iloc[j:i+1], ignore_index=True)
                #break
            #if distance_from_takeoff > 10:
        if data.aALTBARO[i] > 25000: 
            df = df.append(data.iloc[j:i+1], ignore_index=True)
            break
        

#new_df = df[['Time','aALTBARO','aCAS','id']].copy()
#new_df['specEnergyGrad'] = pd.Series(specificEnergyGradient)
df.drop(columns=['aLDGDSEL','aMACH','aTHRANG1','aTHRANG2','aFLAP','LATPOS','LONPOS','aGS','aSAP','TRUTRAKANG','aSAT'], axis=1,inplace=True)
df['specEnergyGrad'] = pd.Series(specificEnergyGradient)
#df['aLDGDSEL'] = pd.Series(landing_gear)
#df['specEnergy'] = pd.Series(specificEnergy)
df['flightPathAngle'] = pd.Series(flightPathAngle)
df['dist2takeoff'] = pd.Series(distance_from_takeoff_list)
df['aALTBARO'] = df['aALTBARO'] * ft_to_m
df['aCAS'] = df['aCAS'] * knot_to_ms
# df['aGS'] = df['aGS'] * knot_to_ms
# df['aTHRANG1'] = df['aTHRANG1'] / 100
# df['aTHRANG2'] = df['aTHRANG2'] / 100

labels = pd.Series(y)
df.head()

,Time,aALTBARO,aCAS,id,specEnergyGrad,flightPathAngle,dist2takeoff
0,1514073855,473.9640,101.154337,0,158.017687,0.528988,3.354763
1,1514073856,489.8136,101.154337,0,128.650893,0.426502,3.411801
2,1514073857,504.4440,100.701630,0,126.549490,0.857744,3.432816
3,1514073858,518.1600,100.444410,0,129.793281,0.388832,3.486852
4,1514073859,530.6568,100.573020,0,102.899078,0.373130,3.537887


In [72]:
extraction_settings = ComprehensiveFCParameters()

# features_filtered_direct = extract_relevant_features(df, labels,
#                                                      column_id='id', column_sort='Time')
X = extract_features(df, column_id='id', column_sort='Time',
                     default_fc_parameters=extraction_settings,
                     # we impute = remove all NaN features automatically
                     impute_function=impute)

X.head()

Feature Extraction: 100%|██████████| 40/40 [36:20<00:00, 54.52s/it]   


,aALTBARO__variance_larger_than_standard_deviation,aALTBARO__has_duplicate_max,aALTBARO__has_duplicate_min,aALTBARO__has_duplicate,aALTBARO__sum_values,aALTBARO__abs_energy,aALTBARO__mean_abs_change,aALTBARO__mean_change,aALTBARO__mean_second_derivative_central,aALTBARO__median,...,dist2takeoff__permutation_entropy__dimension_6__tau_1,dist2takeoff__permutation_entropy__dimension_7__tau_1,dist2takeoff__query_similarity_count__query_None__threshold_0.0,"dist2takeoff__matrix_profile__feature_""min""__threshold_0.98","dist2takeoff__matrix_profile__feature_""max""__threshold_0.98","dist2takeoff__matrix_profile__feature_""mean""__threshold_0.98","dist2takeoff__matrix_profile__feature_""median""__threshold_0.98","dist2takeoff__matrix_profile__feature_""25""__threshold_0.98","dist2takeoff__matrix_profile__feature_""75""__threshold_0.98",dist2takeoff__mean_n_absolute_max__number_of_maxima_7
0,1.0,0.0,0.0,0.0,2.986149e+06,1.558943e+10,9.975197,9.812135,-0.005024,3965.6004,...,-0.0,-0.0,0.0,1.409903,45.254834,9.272173,1.483630,1.444428,1.556043,94.556428
1,1.0,0.0,0.0,0.0,3.215858e+06,1.705573e+10,9.420768,9.281792,-0.003558,4143.1464,...,-0.0,-0.0,0.0,1.239948,1.332421,1.294232,1.292311,1.284703,1.305740,95.023854
2,1.0,0.0,0.0,0.0,3.271429e+06,1.694552e+10,8.899024,8.899024,-0.006445,4017.5688,...,-0.0,-0.0,0.0,1.148663,2.129348,1.502708,1.381656,1.261418,1.780223,98.082978
3,1.0,0.0,0.0,0.0,3.553661e+06,1.945364e+10,8.710844,8.695308,-0.002407,4456.1760,...,-0.0,-0.0,0.0,1.022162,1.456504,1.176436,1.174139,1.087372,1.246143,109.386396
4,1.0,0.0,0.0,0.0,3.432698e+06,1.880862e+10,9.135928,9.019082,-0.004041,4554.7788,...,-0.0,-0.0,0.0,1.248505,1.839529,1.551461,1.579943,1.385094,1.719878,102.690071


In [2]:
## Kaydedilen değişkenleri yüklemek için burayı çalıştır

%store -r X_filtered
%store -r X
%store -r labels

In [3]:
""" prev_item = 0
for item in df['id']:
    if item - prev_item > 1:
        print(item)
        break
    else:
        prev_item = item

print(df['id'] == 372) """

new_labels = labels.drop(372) ### X datasında 372. data alınmamış bozuk

    

In [127]:
X_filtered = select_features(X, new_labels)
X_filtered.head()

,aCAS__quantile__q_0.2,aCAS__quantile__q_0.3,"aALTBARO__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8","aALTBARO__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8",aCAS__abs_energy,"aALTBARO__fft_aggregated__aggtype_""skew""","aALTBARO__fft_aggregated__aggtype_""kurtosis""","aALTBARO__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6","aALTBARO__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.6",aALTBARO__abs_energy,...,"flightPathAngle__fft_coefficient__attr_""angle""__coeff_43","flightPathAngle__fft_coefficient__attr_""real""__coeff_55","flightPathAngle__fft_coefficient__attr_""real""__coeff_82","flightPathAngle__fft_coefficient__attr_""imag""__coeff_13","aCAS__fft_coefficient__attr_""real""__coeff_53","flightPathAngle__fft_coefficient__attr_""imag""__coeff_67","aCAS__fft_coefficient__attr_""angle""__coeff_81","specEnergyGrad__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""min""","specEnergyGrad__fft_coefficient__attr_""imag""__coeff_56","flightPathAngle__fft_coefficient__attr_""angle""__coeff_41"
0,131.799528,133.221440,8.061434,8.061434,1.663794e+07,2.345812,7.929650,9.711703,9.711703,1.558943e+10,...,-100.454907,-0.438042,-3.977741,2.408658,-37.730333,-1.695923,108.380455,0.123209,654.017641,-58.675455
1,133.049617,134.578533,7.523018,7.523018,1.764816e+07,2.370554,8.051635,8.297883,8.297883,1.705573e+10,...,83.459260,-0.251129,-3.009937,5.740244,-45.107676,-1.092215,109.955141,0.101051,-267.826259,-10.164385
2,129.834367,136.231429,8.000527,8.000527,1.807962e+07,2.350209,7.951014,8.750221,8.750221,1.694552e+10,...,-75.899126,-2.501994,3.035477,16.839913,-25.661864,-2.330179,103.078555,0.046890,-291.474787,-66.039930
3,141.781722,144.171810,6.571785,6.571785,1.872834e+07,2.403699,8.216260,6.302597,6.302597,1.945364e+10,...,-91.733326,-0.146542,-0.605736,-0.570587,-39.205402,3.639543,104.443303,0.058611,-914.075961,10.634306
4,134.887197,137.985669,6.275408,6.275408,1.770565e+07,2.398522,8.189725,6.550796,6.550796,1.880862e+10,...,-156.175382,2.247851,-0.076903,-1.351689,-22.964057,0.327293,114.206712,0.112751,427.011742,-6.783766


In [11]:
X_full_train, X_full_test, y_train, y_test = train_test_split(X_filtered, new_labels, test_size=.25)
regressor = ExtraTreesRegressor()
regressor.fit(X_full_train, y_train)
accuracy = regressor.score(X_full_test, y_test)
print(accuracy)


0.9481834069742942


In [12]:
takeoff_weight_estimation = regressor.predict(X_full_test)
takeoff_weight_estimation_percentage = takeoff_weight_estimation / ac.reference_mass*100
takeoff_weight_percentage = y_test / ac.reference_mass*100
takeoff_weight_error = (y_test - takeoff_weight_estimation)/ac.reference_mass*100
mean_takeoff_weight_error = np.mean(takeoff_weight_error)
std_takeoff_weight_error = np.std(takeoff_weight_error)

print("Actual Weights Mean: ", np.mean(y_test))
print("Actual Weights Standard Deviation: ", np.std(y_test))
print("Actual Weights Max: ", np.max(y_test))
print("Actual Weights Min: ", np.min(y_test))
print("Estimated Weights Mean: ", np.mean(takeoff_weight_estimation))
print("Estimated Weights Standard Deviation: ", np.std(takeoff_weight_estimation))
print("Estimated Weights Max: ", np.max(takeoff_weight_estimation))
print("Estimated Weights Min: ", np.min(takeoff_weight_estimation))

Actual Weights Mean:  312752.7021176471
Actual Weights Standard Deviation:  35541.54235962679
Actual Weights Max:  350686.9792
Actual Weights Min:  215838.2016
Estimated Weights Mean:  312833.6455276863
Estimated Weights Standard Deviation:  34734.05933603536
Estimated Weights Max:  343734.47604799987
Estimated Weights Min:  218778.6159199999


In [13]:
%matplotlib qt
fig1 = plt.figure(figsize=[7, 5])
x = range(0,100)
plt.plot(x,x, linewidth=.5, color = 'r')
plt.scatter(takeoff_weight_percentage, takeoff_weight_estimation_percentage, s = 2, color = 'k')
plt.title('Take-off Weight Estimation', fontsize=12, y=1.1)
#plt.xlim(50, 100)
#plt.ylim(50, 100)
plt.ylabel('MTOW(%) Estimated')
plt.xlabel('MTOW(%) Actual')
plt.grid()

fig2, ax2 = plt.subplots()
plt.hist(takeoff_weight_error, bins=26, edgecolor = "black")
plt.title('Take-off Weight Estimation Error', fontsize=12, y=1.1)
plt.ylabel('Number of Aircraft')
plt.xlabel('MTOW(%) Error')
props = dict(boxstyle='square', facecolor='cyan', alpha=0.5)
textstr =  f'Mean Absolute Error: {np.abs(mean_takeoff_weight_error):.2f}%MTOW\n Standart Deviation:{std_takeoff_weight_error:.2f}%MTOW'
plt.text(0.05, 0.95, textstr, transform = ax2.transAxes, fontsize = 8, verticalalignment ='top', bbox = props)
plt.xlim(-40, 40)

(-40.0, 40.0)

In [134]:
### İşlemler uzun sürüyor o yüzden önemli değişkenleri kaydediyoruz

%store X_filtered
%store labels 
%store X 

Stored 'X_filtered' (DataFrame)
Stored 'labels' (Series)
Stored 'X' (DataFrame)
